In [14]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

In [15]:
# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

In [16]:
# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [17]:
# Given order
order = [
    "Актуализация", "Внебаланс (new)", "ДИЗ", "ДИЗ (возражения)", "ДИЗ (чередование)",
    "Коллекшн ранний сбор", "Матрица мотиваторов", "Матрица мотиваторов (распознавание эмоций)",
    "Матрица мотиваторов Хард. Ранний сбор", "Матрица мотиваторов Хард. Ранний сбор (кредитные карты)",
    "Матрица мотивации. Выезд", "Мультидоговоры. Универсальный", "Неконтактные", "Неконтактные (внебаланс)",
    "Неконтактные (поздний сбор)", "Неконтактные Суд", "Ожидание выезда", "Ожидание суда",
    "Подготовка к передаче в КА", "Преколлекшн (rename)", "Суд", "Усиление ПС Предцессия", 
    "Усиление позднего сбора", "ФЗ-230"
]
# Given column order
column_order = [
    "АО: Абонент не отвечает", "АО: Абонент недоступен", "АО: Номер не существует", 
    "АО: Нужен внутренний номер", "АО: Соединение установлено", "АО: Умные голосовые помощники", 
    "АО: Факс", "Должник молчит", "Должник не уверен", "Должник неизвестен", "Должник умер", 
    "Запрос реструктуризации", "Заявление о факте платежа", "Клиент болен", "Не передадут информацию", 
    "Не пройдена верификация", "Неконструктивный диалог", "Нужна помощь оператора", "Нужна помощь оператора *", 
    "Обещание оплатить", "Обещание частичной оплаты", "Оплата по испол.листу", "Отказ от верификации", 
    "Отказ от оплаты", "Отрицает долг", "Перезвонить", "Просьба передать информацию", "Сброс звонка роботом", 
    "Связь прервалась"
]

In [18]:

# 1. Filter the dataframe
filtered_df = df[df['Результат автооценки'] != 100] # !!!
#filtered_df = df # !!!
# 2. Create the pivot table
pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='size', 
                                      fill_value=0)

'''pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='count', 
                                      fill_value=0)'''

pivot_table_mean = df.pivot_table(index='Имя колл-листа', columns='Результат робота', values='Результат автооценки', aggfunc='mean')


'''# 3. Ensure all desired columns are present
for col in column_order:
    if col not in pivot_table.columns:
        pivot_table[col] = 0

# Ensure all desired columns are present in the mean pivot table
for col in column_order:
    if col not in pivot_table_mean.columns:
        pivot_table_mean[col] = 0'''


'''# Reorder columns based on the given order
pivot_table = pivot_table[column_order]
pivot_table_mean = pivot_table_mean[column_order]
# Assuming pivot_table is the pivot table you previously created
pivot_table = pivot_table[column_order]
pivot_table = pivot_table.reindex(order)
pivot_table_mean = pivot_table_mean.reindex(order)'''

'# Reorder columns based on the given order\npivot_table = pivot_table[column_order]\npivot_table_mean = pivot_table_mean[column_order]\n# Assuming pivot_table is the pivot table you previously created\npivot_table = pivot_table[column_order]\npivot_table = pivot_table.reindex(order)\npivot_table_mean = pivot_table_mean.reindex(order)'

In [19]:
pivot_table

Результат робота,АО: Умные голосовые помощники,Должник не уверен,Должник неизвестен,Запрос реструктуризации,Заявление о факте платежа,Клиент болен,Не передадут информацию,Не пройдена верификация,Неконструктивный диалог,Нужна помощь оператора,Нужна помощь оператора *,Обещание оплатить,Обещание частичной оплаты,Оплата по испол.листу,Отказ от верификации,Отказ от оплаты,Отрицает долг,Перезвонить,Просьба передать информацию,Связь прервалась
Имя колл-листа,,,,,,,,,,,,,,,,,,,,
Актуализация,1,0,1,0,0,1,4,1,0,1,0,0,0,0,1,0,0,0,4,0
Внебаланс (new),20,0,56,0,9,6,47,1,92,86,0,80,29,6,9,144,5,14,90,0
ДИЗ,9,0,13,0,1,1,7,0,1,22,0,9,0,4,8,28,2,5,12,1
ДИЗ (возражения),1,0,2,0,0,0,1,0,0,4,0,1,0,0,0,4,0,0,2,0
ДИЗ (чередование),0,0,2,0,0,1,0,0,0,0,0,1,0,0,0,4,0,0,0,0
Коллекшн ранний сбор,1,0,14,0,23,1,11,10,0,11,0,52,0,0,19,16,2,4,30,4
Матрица мотиваторов,6,0,27,5,88,6,48,0,57,127,5,1665,0,0,29,460,50,19,110,3
Матрица мотиваторов (распознавание эмоций),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
Матрица мотиваторов Хард. Ранний сбор,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [20]:
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill
import xlsxwriter


def format_xlsx(pivot_table:pd.core.frame.DataFrame, sheet:str = 'Отчет 1', name:str = "pivot_table_gradient_colorscale.xlsx"):
    # Create an Excel writer and export the pivot table to an Excel file
    excel_file_path = name
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        pivot_table.to_excel(writer, sheet_name=sheet, index=True)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet]

        # Get the dimensions of the pivot table
        max_row = len(pivot_table)
        max_col = len(pivot_table.columns)

        # Add a format for the header cells
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'border': 1, 'bg_color': '#EFEFEF', 'align': 'center'})

        # Set the column width and format for the header
        for col_num, value in enumerate(pivot_table.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
            column_len = max(pivot_table[value].astype(str).str.len().max(), len(value)) + 2
            worksheet.set_column(col_num + 1, col_num + 1, column_len)
        
        # Add a format for the value cells
        value_format = workbook.add_format({'border': None, 'valign': 'vcenter', 'align': 'center',})

        # Get Max and Min of the value FFS
        max_value = pivot_table.max().max()
        min_value = pivot_table.min().min()


        # Apply gradient color scale to value cells
        worksheet.conditional_format(1, 1, max_row, max_col, {
            'type': '3_color_scale',
            'min_color': '#A6D86E',  # Green
            'mid_color': '#FFFFFE',  # White (for NaN)
            'max_color': '#e85f5f',  # Red
            'min_type': 'num',
            'mid_type': 'num',
            'max_type': 'num'
        })
        print(f'file: {name} -- Transformed 0')

In [21]:
format_xlsx(pivot_table)

PermissionError: [Errno 13] Permission denied: 'pivot_table_gradient_colorscale.xlsx'

# Динамика по кол-листам + статусам + 3 в 1

In [22]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [25]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [27]:
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [29]:
# Using pivot_table to create the pivot
# pivot_df = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean')
pivot_df = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
# If you want to handle missing values (NaN) gracefully, you can replace them with 0
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.replace(0.00, '')

In [30]:
pivot_df

Дата,02.08.2023,03.08.2023,31.07.2023
Имя колл-листа,,,
Актуализация,1.0,4.0,9.0
Внебаланс (new),199.0,213.0,282.0
Внебаланс (new) (тест контур API),,,
ДИЗ,34.0,38.0,51.0
ДИЗ (возражения),4.0,5.0,6.0
ДИЗ (тест контур API),,,
ДИЗ (чередование),1.0,2.0,5.0
Коллекшн ранний сбор,51.0,74.0,73.0
Матрица мотиваторов,680.0,1276.0,749.0


In [34]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [35]:
pivot_df = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum')
# If you want to handle missing values (NaN) gracefully, you can replace them with 0
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.replace(0.00, '')
pivot_df

Дата                                         02.08.2023 03.08.2023 31.07.2023
Имя колл-листа Результат робота                                              
Актуализация   АО: Абонент недоступен                                        
               АО: Номер не существует                                       
               АО: Нужен внутренний номер                                    
               АО: Соединение установлено                                    
               АО: Умные голосовые помощники                              1.0
...                                                 ...        ...        ...
ФЗ-230         Отрицает долг                                             18.0
               Перезвонить                                               13.0
               Просьба передать информацию                               50.0
               Сброс звонка роботом                                          
               Связь прервалась                                           1.0

[439 rows x 3 columns]

In [36]:
# Assuming you have a dataframe named 'df'
# Creating a pivot table to calculate the count of Ошибки (True values) and the count of Результат робота
pivot_df = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum')

# Calculate the count of Результат робота for each combination
count_df = df.groupby(['Имя колл-листа', 'Результат робота', 'Дата'])['Результат робота'].count().unstack()

# Calculating the fraction as a string
pivot_df = (pivot_df / count_df).applymap(lambda x: x if not pd.isna(x) else '')

pivot_df

Дата                                         02.08.2023 03.08.2023 31.07.2023
Имя колл-листа Результат робота                                              
Актуализация   АО: Абонент недоступен               0.0        0.0        0.0
               АО: Номер не существует                         0.0        0.0
               АО: Нужен внутренний номер           0.0        0.0        0.0
               АО: Соединение установлено           0.0        0.0        0.0
               АО: Умные голосовые помощники        0.0        0.0   0.002703
...                                                 ...        ...        ...
ФЗ-230         Отрицает долг                                         0.147541
               Перезвонить                                           0.036827
               Просьба передать информацию          0.0        0.0    0.02004
               Сброс звонка роботом                                       0.0
               Связь прервалась                     0.0        0.0   0.000108

[439 rows x 3 columns]